In [4]:
# import libraries

import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [31]:
cap.release()
cv2.destroyAllWindows()

In [6]:
# index of landmarks in right and left eye

left_eye_lm = [130, 247, 30, 29, 27, 28, 56, 190, 243, 112, 26, 22, 23, 24, 110, 25]
right_eye_lm = [359, 467, 260, 259, 257, 258, 286, 414, 463, 341, 256, 252, 253, 254, 339, 255]

In [38]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        height, width, _ = frame.shape
        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        frame.flags.writeable = False
        results = holistic.process(frame)

        # get eyes coords
        #left_eye = eye_coords(frame, left_eye_lm)
        #right_eye = eye_coords(frame, right_eye_lm)
        left_eye_x = []
        left_eye_y = []
        for lm in left_eye_lm:
            # mp return normalized coords, so we multiply
            left_eye_x.append(int((results.face_landmarks.landmark[lm].x) * width))
            left_eye_y.append(int((results.face_landmarks.landmark[lm].y) * height))
        left_eye = list(zip(left_eye_x, left_eye_y))

        right_eye_x = []
        right_eye_y = []
        for lm in right_eye_lm:
            right_eye_x.append(int((results.face_landmarks.landmark[lm].x) * width))
            right_eye_y.append(int((results.face_landmarks.landmark[lm].y) * height))
        right_eye = list(zip(right_eye_x, right_eye_y))
        left_eye = np.array(left_eye, dtype="int")
        right_eye = np.array(right_eye, dtype="int")

        # find convex hull for the eyes   
        leye_hull = cv2.convexHull(left_eye)
        reye_hull = cv2.convexHull(right_eye)

        # Draw convex hull
        frame.flags.writeable = True
        cv2.drawContours(frame, [leye_hull], -1, (0, 255, 0), 2)
        cv2.drawContours(frame, [reye_hull], -1, (0, 255, 0), 2)

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        cv2.imshow('Webcam Feed', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()